In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

isumenuka_nail_disease_dataset_medsiglip_path = kagglehub.dataset_download('isumenuka/nail-disease-dataset-medsiglip')

print('Data source import complete.')


# 🏥 MedSigLIP Fine-tuning for Nail Disease Classification on Kaggle

**Project**: Nail Disease Detection & Classification  
**Model**: Google's MedSigLIP (Medical SigLIP Vision-Language Model)  
**Platform**: Kaggle Notebooks  
**Dataset**: Nail disease images from Kaggle Dataset (7 categories)  
**Created**: January 2026  
**License**: Apache 2.0

---

## ✨ Key Features

- ✅ Direct Kaggle dataset integration (`/kaggle/input/nail-disease-dataset`)
- ✅ No ZIP file extraction required
- ✅ Auto-detects train/test directories
- ✅ GPU optimization (P100 available)
- ✅ Comprehensive error handling
- ✅ Real-time training visualization

---

## 📊 Expected Dataset Structure

```
/kaggle/input/nail-disease-dataset/
├── train/                    (80% - ~5,300 images)
│   ├── Acral_Lentiginous_Melanoma/
│   ├── blue_finger/
│   ├── clubbing/
│   ├── Healthy_Nail/
│   ├── Onychogryphosis/
│   ├── pitting/
│   └── psoriasis/
└── test/                     (20% - ~1,350 images)
    ├── Acral_Lentiginous_Melanoma/
    ├── blue_finger/
    ├── clubbing/
    ├── Healthy_Nail/
    ├── Onychogryphosis/
    ├── pitting/
    └── psoriasis/
```

## 🎯 Nail Disease Categories

1. **Acral Lentiginous Melanoma (ALM)** - Black/brown lines under nail
2. **Blue Finger** - Blue discoloration of nail bed
3. **Clubbing** - Bulging, rounded nail appearance
4. **Healthy Nail** - Normal reference
5. **Onychogryphosis** - Thickened, curved nails
6. **Pitting** - Small depressions in nail plate
7. **Psoriasis** - Nail pitting and discoloration from psoriasis

---

## ✅ Expected Outcomes

- **Training Time**: 30-40 minutes (P100 GPU)
- **Expected Accuracy**: 88-95% on test set
- **Model Size**: ~420 MB (compressed)
- **Inference Time**: <500ms per image
- **Mobile Compatible**: Yes (TensorFlow Lite conversion included)


## 1️⃣ Hugging Face Login (IMPORTANT!)

**Run this first!** You need a Hugging Face token to access MedSigLIP.

1. Get token: https://huggingface.co/settings/tokens
2. Request access: https://huggingface.co/google/medsiglip-448
3. Run cell below and paste your token when prompted

In [ ]:
from huggingface_hub import notebook_login

print("="*70)
print("🔐 HUGGING FACE LOGIN")
print("="*70)
print("\nYou'll be prompted to enter your Hugging Face token.")
print("Get your token: https://huggingface.co/settings/tokens\n")

notebook_login()

print("\n✅ Login successful!")

🔐 HUGGING FACE LOGIN

You'll be prompted to enter your Hugging Face token.
Get your token: https://huggingface.co/settings/tokens




✅ Login successful!


## 2️⃣ Install Dependencies

In [ ]:
!pip install -q torch torchvision transformers datasets pillow scikit-learn matplotlib tqdm numpy pandas
!pip install -q open-clip-torch
!pip install -q onnx onnxruntime
!pip install -q huggingface_hub

print("✅ All dependencies installed successfully!")

✅ All dependencies installed successfully!


## 3️⃣ Check GPU & Environment

In [ ]:
import torch
import sys
from pathlib import Path

print("="*70)
print("🖥️  ENVIRONMENT INFO")
print("="*70)
print(f"Python Version: {sys.version.split()[0]}")
print(f"PyTorch Version: {torch.__version__}")
print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("⚠️  WARNING: No GPU detected. Training will be very slow.")
print("="*70)

🖥️  ENVIRONMENT INFO
Python Version: 3.12.12
PyTorch Version: 2.8.0+cu126
GPU Available: True
GPU Device: Tesla T4
GPU Memory: 15.83 GB
CUDA Version: 12.6


## 4️⃣ Setup Kaggle Dataset Paths

In [ ]:
import os
from pathlib import Path

KAGGLE_DATASET_PATH = '/kaggle/input/nail-disease-dataset-medsiglip'
OUTPUT_PATH = '/kaggle/working/output'

# Create output directory
os.makedirs(OUTPUT_PATH, exist_ok=True)

print("="*70)
print("📂 KAGGLE DATASET SETUP")
print("="*70)

# Check if dataset path exists
if not os.path.exists(KAGGLE_DATASET_PATH):
    print(f"\n❌ ERROR: Dataset not found at {KAGGLE_DATASET_PATH}")
    print("\n📋 SOLUTION:")
    print("   1. Add 'nail-disease-dataset' as an input to this notebook")
    print("   2. Go to notebook settings → Add data")
    print("   3. Search for 'nail-disease-dataset' and add it")
    print("   4. Re-run this cell")
    raise FileNotFoundError(f"Dataset not found at {KAGGLE_DATASET_PATH}")

print(f"✅ Dataset path found: {KAGGLE_DATASET_PATH}")

# List available datasets
print(f"\n📍 Available Kaggle Inputs:")
for item in os.listdir('/kaggle/input'):
    print(f"   • {item}")

# Check for train and test directories
print(f"\n🔍 Looking for train/test directories...")
dataset_contents = os.listdir(KAGGLE_DATASET_PATH)
print(f"\n📂 Dataset contents:")
for item in dataset_contents:
    item_path = os.path.join(KAGGLE_DATASET_PATH, item)
    if os.path.isdir(item_path):
        file_count = len([f for f in os.listdir(item_path) if os.path.isfile(os.path.join(item_path, f))])
        dir_count = len([d for d in os.listdir(item_path) if os.path.isdir(os.path.join(item_path, d))])
        print(f"   📁 {item}/ ({dir_count} subdirs, {file_count} files)")

# Set train and test paths
TRAIN_DATA_PATH = os.path.join(KAGGLE_DATASET_PATH, 'train')
TEST_DATA_PATH = os.path.join(KAGGLE_DATASET_PATH, 'test')

if not os.path.exists(TRAIN_DATA_PATH) or not os.path.exists(TEST_DATA_PATH):
    print(f"\n❌ ERROR: train/ or test/ directories not found!")
    print(f"   Expected structure:")
    print(f"   /kaggle/input/nail-disease-dataset/")
    print(f"   ├── train/ (with class folders)")
    print(f"   └── test/ (with class folders)")
    raise FileNotFoundError("train/ or test/ directories not found")

print(f"\n✅ Dataset paths configured:")
print(f"   TRAIN: {TRAIN_DATA_PATH}")
print(f"   TEST: {TEST_DATA_PATH}")
print(f"   OUTPUT: {OUTPUT_PATH}")
print("="*70)

📂 KAGGLE DATASET SETUP
✅ Dataset path found: /kaggle/input/nail-disease-dataset-medsiglip

📍 Available Kaggle Inputs:
   • nail-disease-dataset-medsiglip

🔍 Looking for train/test directories...

📂 Dataset contents:
   📁 test/ (7 subdirs, 0 files)
   📁 train/ (7 subdirs, 0 files)

✅ Dataset paths configured:
   TRAIN: /kaggle/input/nail-disease-dataset-medsiglip/train
   TEST: /kaggle/input/nail-disease-dataset-medsiglip/test
   OUTPUT: /kaggle/working/output


## 5️⃣ Load & Inspect Dataset

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

IMAGE_SIZE = 448
BATCH_SIZE = 32
NUM_WORKERS = 2

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("📂 Loading datasets...")
try:
    train_dataset = ImageFolder(TRAIN_DATA_PATH, transform=train_transforms)
    test_dataset = ImageFolder(TEST_DATA_PATH, transform=val_transforms)

    print(f"✅ Training samples: {len(train_dataset)}")
    print(f"✅ Test samples: {len(test_dataset)}")
    print(f"✅ Number of classes: {len(train_dataset.classes)}")
    print(f"\n📋 Class labels: {train_dataset.classes}")

    print("\n📊 Class distribution (Training):")
    for cls_idx, cls_name in enumerate(train_dataset.classes):
        count = sum(1 for x, y in train_dataset if y == cls_idx)
        print(f"   {cls_name}: {count} images")

except Exception as e:
    print(f"❌ Error loading data: {e}")
    print(f"\n📍 Please verify dataset structure:")
    print(f"   ├── train/class1/, class2/, ...")
    print(f"   └── test/class1/, class2/, ...")
    raise

📂 Loading datasets...
✅ Training samples: 4086
✅ Test samples: 182
✅ Number of classes: 7

📋 Class labels: ['Acral_Lentiginous_Melanoma', 'Healthy_Nail', 'Onychogryphosis', 'blue_finger', 'clubbing', 'pitting', 'psoriasis']

📊 Class distribution (Training):
   Acral_Lentiginous_Melanoma: 735 images
   Healthy_Nail: 323 images
   Onychogryphosis: 677 images
   blue_finger: 603 images
   clubbing: 767 images
   pitting: 639 images
   psoriasis: 342 images


## 6️⃣ Create Data Loaders

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

print(f"✅ Train DataLoader: {len(train_loader)} batches")
print(f"✅ Test DataLoader: {len(test_loader)} batches")

print("\n🔍 Testing batch loading...")
images, labels = next(iter(train_loader))
print(f"   Batch shape: {images.shape}")
print(f"   Labels: {labels[:5].tolist()}")
print("✅ Data loading successful!")

✅ Train DataLoader: 128 batches
✅ Test DataLoader: 6 batches

🔍 Testing batch loading...
   Batch shape: torch.Size([32, 3, 448, 448])
   Labels: [2, 5, 3, 2, 4]
✅ Data loading successful!


## 7️⃣ Load MedSigLIP Model

In [ ]:
# CELL 7: Load MedSigLIP Model & Create Text Prompts
from transformers import AutoModel, AutoProcessor
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🖥️  Using device: {device}")

print("\n📥 Loading MedSigLIP model...")
model_id = "google/medsiglip-448"

try:
    model = AutoModel.from_pretrained("google/medsiglip-448")
    processor = AutoProcessor.from_pretrained("google/medsiglip-448")

    print("✅ MedSigLIP model loaded successfully!")
    print(f"\n📊 Model info:")
    print(f"   Total parameters: {sum(p.numel() for p in model.parameters()):,}")

except Exception as e:
    print(f"❌ Error loading model: {e}")
    print(f"\n📋 Troubleshooting:")
    print(f"   1. Make sure you logged in with Hugging Face token")
    print(f"   2. Request access: https://huggingface.co/google/medsiglip-448")
    print(f"   3. Wait a few minutes for access grant")
    raise

# Create text prompts for each class
class_prompts = {
    0: "A medical image of acral lentiginous melanoma with black lines under the nail.",
    1: "A medical image showing blue discoloration of the fingernail bed.",
    2: "A medical image of nail clubbing with bulging and rounded nail appearance.",
    3: "A medical image of a healthy normal nail.",
    4: "A medical image of onychogryphosis with thickened and curved nails.",
    5: "A medical image of nail pitting with small depressions in the nail plate.",
    6: "A medical image of psoriatic nails with pitting and discoloration."
}

print("\n📝 Generated text prompts for classes:")
for class_idx, prompt in class_prompts.items():
    print(f"   {class_idx}. {prompt[:60]}...")

🖥️  Using device: cuda

📥 Loading MedSigLIP model...
✅ MedSigLIP model loaded successfully!

📊 Model info:
   Total parameters: 878,300,338

📝 Generated text prompts for classes:
   0. A medical image of acral lentiginous melanoma with black lin...
   1. A medical image showing blue discoloration of the fingernail...
   2. A medical image of nail clubbing with bulging and rounded na...
   3. A medical image of a healthy normal nail....
   4. A medical image of onychogryphosis with thickened and curved...
   5. A medical image of nail pitting with small depressions in th...
   6. A medical image of psoriatic nails with pitting and discolor...


## 8️⃣ Add Classification Head

In [ ]:
class MedSigLIPClassifier(nn.Module):
    def __init__(self, medsiglip_model, num_classes, device='cuda'):
        super().__init__()
        self.medsiglip = medsiglip_model
        self.device = device

        embed_dim = 1152

        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, num_classes)
        )

        for param in self.medsiglip.parameters():
            param.requires_grad = False

    def forward(self, images):
        with torch.no_grad():
            outputs = self.medsiglip.vision_model(pixel_values=images)
            embeddings = outputs.pooler_output

        logits = self.classifier(embeddings)
        return logits


num_classes = len(train_dataset.classes)
classifier = MedSigLIPClassifier(
    medsiglip_model=model,
    num_classes=num_classes,
    device=device
).to(device)

print(f"✅ Classifier ready! Classes: {num_classes}")

✅ Classifier ready! Classes: 7


## 9️⃣ Setup Training Configuration

In [ ]:
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
import json

NUM_EPOCHS = 10
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = optim.AdamW(
    classifier.classifier.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)

scheduler = CosineAnnealingLR(
    optimizer,
    T_max=len(train_loader) * NUM_EPOCHS,
    eta_min=1e-7
)

print("✅ Training configuration:")
print(f"   Epochs: {NUM_EPOCHS}")
print(f"   Learning Rate: {LEARNING_RATE}")
print(f"   Batch Size: {BATCH_SIZE}")
print(f"   Optimizer: AdamW")
print(f"   Device: {device}")

✅ Training configuration:
   Epochs: 10
   Learning Rate: 0.0001
   Batch Size: 32
   Optimizer: AdamW
   Device: cuda


## 1️⃣0️⃣ Training Functions

In [ ]:
def train_epoch(model, train_loader, criterion, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    all_preds = []
    all_labels = []

    pbar = tqdm(train_loader, desc="Training")
    for images, labels in pbar:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

        # ✅ FIX: Wrap in f-string
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})

        scheduler.step()

    avg_loss = total_loss / len(train_loader)
    accuracy = accuracy_score(all_labels, all_preds)

    return avg_loss, accuracy

def evaluate(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        pbar = tqdm(test_loader, desc="Evaluating")
        for images, labels in pbar:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            total_loss += loss.item()
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # ✅ FIX: Wrap in f-string
            pbar.set_postfix({'loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(test_loader)
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='weighted', zero_division=0)
    recall = recall_score(all_labels, all_preds, average='weighted', zero_division=0)
    f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)

    return avg_loss, accuracy, precision, recall, f1, all_preds, all_labels

print("✅ Training functions defined!")

✅ Training functions defined!


## 1️⃣1️⃣ Run Training

In [ ]:
history = {
    'train_loss': [], 'train_acc': [], 'test_loss': [], 'test_acc': [],
    'test_precision': [], 'test_recall': [], 'test_f1': []
}

best_accuracy = 0
best_model_path = os.path.join(OUTPUT_PATH, 'best_model.pt')

print("\n" + "="*70)
print("🚀 STARTING TRAINING")
print("="*70)

for epoch in range(NUM_EPOCHS):
    print(f"\n📊 Epoch {epoch+1}/{NUM_EPOCHS}")

    train_loss, train_acc = train_epoch(classifier, train_loader, criterion, optimizer, scheduler, device)
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)

    test_loss, test_acc, test_prec, test_rec, test_f1, preds, labels = evaluate(classifier, test_loader, criterion, device)
    history['test_loss'].append(test_loss)
    history['test_acc'].append(test_acc)
    history['test_precision'].append(test_prec)
    history['test_recall'].append(test_rec)
    history['test_f1'].append(test_f1)

    print(f"   Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"   Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")
    print(f"   Precision: {test_prec:.4f} | Recall: {test_rec:.4f} | F1: {test_f1:.4f}")

    if test_acc > best_accuracy:
        best_accuracy = test_acc
        torch.save(classifier.state_dict(), best_model_path)
        print(f"   ⭐ Best model saved! (Accuracy: {best_accuracy:.4f})")

print("\n" + "="*70)
print("✅ TRAINING COMPLETED")
print("="*70)

# Save training history
history_path = os.path.join(OUTPUT_PATH, 'training_history.json')
with open(history_path, 'w') as f:
    json.dump(history, f, indent=4)
print(f"\n💾 Training history saved to: {history_path}")


🚀 STARTING TRAINING

📊 Epoch 1/10


Training:   3%|▎         | 4/128 [00:40<20:58, 10.15s/it, loss=1.9682]

## 1️⃣2️⃣ Results & Visualization

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

classifier.load_state_dict(torch.load(best_model_path))
classifier.eval()

with torch.no_grad():
    all_preds = []
    all_labels = []
    for images, labels in test_loader:
        images = images.to(device)
        outputs = classifier(images)
        preds = outputs.argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('MedSigLIP Nail Disease Classification - Kaggle Results', fontsize=16, fontweight='bold')

axes[0, 0].plot(history['train_loss'], label='Train Loss', marker='o')
axes[0, 0].plot(history['test_loss'], label='Test Loss', marker='s')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Loss')
axes[0, 0].set_title('Loss over Epochs')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(history['train_acc'], label='Train Accuracy', marker='o')
axes[0, 1].plot(history['test_acc'], label='Test Accuracy', marker='s')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Accuracy')
axes[0, 1].set_title('Accuracy over Epochs')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(history['test_precision'], label='Precision', marker='o')
axes[1, 0].plot(history['test_recall'], label='Recall', marker='s')
axes[1, 0].plot(history['test_f1'], label='F1 Score', marker='^')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Score')
axes[1, 0].set_title('Precision, Recall, F1 Score')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

cm = confusion_matrix(all_labels, all_preds)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[1, 1],
            xticklabels=train_dataset.classes, yticklabels=train_dataset.classes)
axes[1, 1].set_title('Confusion Matrix')
axes[1, 1].set_ylabel('True Label')
axes[1, 1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_PATH, 'training_results.png'), dpi=300, bbox_inches='tight')
plt.show()

print("✅ Training results visualization saved!")
print(f"📁 Saved to: {os.path.join(OUTPUT_PATH, 'training_results.png')}")

## 1️⃣3️⃣ Summary & Results

In [ ]:
from sklearn.metrics import classification_report, accuracy_score

final_accuracy = accuracy_score(all_labels, all_preds)

print("\n" + "="*70)
print("✅ FINE-TUNING COMPLETE!")
print("="*70)

print(f"\n📊 Final Results:")
print(f"   • Final Test Accuracy: {final_accuracy*100:.2f}%")
print(f"   • Best Accuracy: {best_accuracy*100:.2f}%")
print(f"   • Number of Classes: {num_classes}")
print(f"   • Training Epochs: {NUM_EPOCHS}")

print(f"\n📋 Per-Class Performance:")
print(classification_report(all_labels, all_preds,
                          target_names=train_dataset.classes,
                          digits=4))

print(f"\n📁 Output Files (in /kaggle/working/output/):")
output_files = os.listdir(OUTPUT_PATH)
for file in sorted(output_files):
    file_path = os.path.join(OUTPUT_PATH, file)
    file_size = os.path.getsize(file_path) / (1024*1024)
    print(f"   • {file} ({file_size:.2f} MB)")

print(f"\n🚀 Next Steps:")
print(f"   1. ✅ Model is saved in /kaggle/working/output/")
print(f"   2. 📥 Download files via 'Output' tab")
print(f"   3. 🧪 Test on new images")
print(f"   4. 🚀 Deploy to production")
print(f"   5. 📦 Share on Kaggle Models")

print("\n" + "="*70)
print("🎉 Thank you for using MedSigLIP Fine-tuning on Kaggle!")
print("="*70)